<h1><center>МЕТОДЫ ЛОГИРОВАНИЯ МОДЕЛИ И ЕЁ ОКРУЖЕНИЯ</h1></center>

Начнём с простого способа логирования модели в MLflow. Оно происходит следующим образом:

In [5]:

import mlflow

experiment_id = '4'

with mlflow.start_run(run_name='log_model', experiment_id=experiment_id) as run:
    run_id = run.info.run_id
  
    model_info = mlflow.catboost.log_model(cb_model=model, artifact_path="models") 

MlflowException: Could not find experiment with ID 4

Как убедиться, что ваша модель была залогирована корректно? Попробуйте загрузить её из объектного хранилища при помощи функции load_model, в качестве параметра передав model_uri, — его можно получить из переменной model_info, в которой хранится вся информация о логировании конкретной модели в конкретном запуске. Например, сделать это можно так: 

In [ ]:
loaded_model = mlflow.catboost.load_model(model_uri=model_info.model_uri)

**Регистрация модели**

Обычно полное сохранение модели осуществляется при её регистрации. 
<br>Регистрация модели отличается от простого логирования тем, что помещает модель в Model Registry. Это даёт возможность версионирования модели, установки тегов, назначения различных статусов и других важных атрибутов.
Чтобы поместить что-то в Model Registry, необходимо с помощью кода подключиться к этому модулю MLflow. Процесс аналогичен подключению к Tracking Server. Как правило, URI Tracking Server и Model Registry совпадают. В нашем случае оба модуля поднимаются локально.



In [6]:
TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}") 

**Чтобы зарегистрировать модель в MLflow**, в функции log_model используют параметр registered_model_name, в котором нужно указать название вашей модели в реестре —  оно должно быть уникальным. При такой регистрации версия модели будет автоматически обновляться на один пункт.
<br>Базовый вариант регистрации модели выглядит следующим образом:

In [7]:
experiment_id = '4'

with mlflow.start_run(run_name='reg_model', experiment_id=experiment_id) as run:

    model_info = mlflow.catboost.log_model(
        cb_model=model,
        artifact_path="models",
        registered_model_name=REGISTRY_MODEL_NAME ) 

NameError: name 'model' is not defined

Вот итоговый код:

In [8]:
import mlflow
import numpy as np

TRACKING_SERVER_HOST = "127.0.0.1"
TRACKING_SERVER_PORT = 5000

# напишите код, который подключает tracking и registry uri
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")
mlflow.set_registry_uri(f"http://{TRACKING_SERVER_HOST}:{TRACKING_SERVER_PORT}")

experiment_id = '0'

# указываем путь до окружения
pip_requirements="../requirements.txt"

# формируем сигнатуру, дополнительно передавая параметры применения модели
signature = mlflow.models.infer_signature(
	np.array([[0.1, 0.2, 0.3], [0.1, 0.2, 0.3]]),
	np.array([0.1, 0.2])
)
# формируем пример входных данных
input_example = [[0.1, 0.2, 0.3], [0.1, 0.2, 0.3]]
# предположим, мы хотим указать на то, что модель предсказывает на месяц вперёд
metadata = {"target_name": "churn"}
# путь до скрипта или ноутбука, который осуществляет обучение модели и валидацию
code_paths = ["train.py", "val_model.py"]


with mlflow.start_run(run_name="model_reg", experiment_id=experiment_id) as run:
    run_id = run.info.run_id
  
    model_info = mlflow.catboost.log_model( model, cb_model=model,
                                           artifact_path='models',
        registered_model_name=REGISTRY_MODEL_NAME,
        pip_requirements=pip_requirements,
        signature=signature,
        input_example=input_example,
        metadata=metadata,
        code_paths=code_paths )

NameError: name 'model' is not defined